In [ ]:
import pathlib

import pandas as pd

import dlsproc.xml
import dlsproc.bundle
import dlsproc.hier
import dlsproc.io

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data


## Insiders

> Licitaciones publicadas en los perfiles del contratante ubicados en la Plataforma de Contratación del Sector Público, excluyendo los contratos menores

A file to *cache* the read data

In [ ]:
insiders_output_file = dlsproc.io.File('insiders')

The directory for *insiders* data

In [ ]:
insiders_directory = data_directory / 'perfiles_plataforma'
assert insiders_directory.exists()
print(insiders_directory)

/home/manu/dlsproc/data/perfiles_plataforma


The data is read from a file if present, or from the given directory if not

In [ ]:
if insiders_output_file.exists():
    insiders_df = insiders_output_file.read()
else:
    # data is read from the given directory, and concatenated into a single `pd.DataFrame`...
    insiders_df = dlsproc.bundle.read_zips_directory(insiders_directory, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    insiders_df = dlsproc.hier.flat_df_to_multiindexed_df(insiders_df)
    
    # ...and saved
    insiders_output_file.write(insiders_df)

In [ ]:
insiders_df.head()

id  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
zip                                                file name                                     entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom 0      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 1      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 2      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 3      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 4      https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                  summary  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                    

How many entries and how many fields?

In [ ]:
insiders_df.shape

(78944, 148)

Examples of columns

In [ ]:
insiders_df.columns.values[10:20]

array([('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyIdentification', 'ID', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'CityName', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'PostalZone', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'AddressLine', 'Line', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'Country', 'IdentificationCode', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'Country', 'Name', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'Contact', 'Name', '', '', ''

The first few rows

In [ ]:
insiders_df.head()

id  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
zip                                                file name                                     entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom 0      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 1      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 2      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 3      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 4      https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                  summary  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                    

The types of the first few columns

In [ ]:
insiders_df.dtypes[:4]

id                                            string
summary                                       string
title                                         string
updated                          datetime64[ns, UTC]
dtype: object

Number of levels in the *columns*

In [ ]:
insiders_df.columns.nlevels

12

*updated* is the 3rd column

In [ ]:
insiders_df.columns[3]

('updated', '', '', '', '', '', '', '', '', '', '', '')

In order to extract it *as a time series*

In [ ]:
insiders_df[insiders_df.columns[3]]

zip                                                  file name                                                      entry
licitacionesPerfilesContratanteCompleto3_202203.zip  licitacionesPerfilesContratanteCompleto3.atom                  0       2022-03-31 20:06:05.604000+00:00
                                                                                                                    1       2022-03-31 19:41:25.548000+00:00
                                                                                                                    2       2022-03-31 19:28:25.157000+00:00
                                                                                                                    3       2022-03-31 19:05:01.018000+00:00
                                                                                                                    4       2022-03-31 19:01:45.736000+00:00
                                                                                             

Without specifying all the levels in the index

In [ ]:
insiders_df['updated']

zip                                                  file name                                                      entry
licitacionesPerfilesContratanteCompleto3_202203.zip  licitacionesPerfilesContratanteCompleto3.atom                  0       2022-03-31 20:06:05.604000+00:00
                                                                                                                    1       2022-03-31 19:41:25.548000+00:00
                                                                                                                    2       2022-03-31 19:28:25.157000+00:00
                                                                                                                    3       2022-03-31 19:05:01.018000+00:00
                                                                                                                    4       2022-03-31 19:01:45.736000+00:00
                                                                                             

### Entity

Columns in the `pd.DataFrame` containing "Name" in their names are displayed

In [ ]:
insiders_df[dlsproc.hier.columns_containing(insiders_df, 'Name')].head()

ContractFolderStatus  \
                                                                                                                                  LocatedContractingParty   
                                                                                                                                                    Party   
                                                                                                                                                PartyName   
                                                                                                                                                     Name   
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
zip                                                file name                                     entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom 0      Alcaldía del Ayuntamiento de Fresno de Cantespino   
                                                                                                 1      Consejo de Administración de ICEX España Expor...   
                                                                                                 2             Alcaldia del Ayuntamiento de Palma del Río   
                                                                                                 3      Junta de Gobierno del Ayuntamiento de Palma de...   
                                                                                                 4      Concejalía de Personal y Contratación del Ayun...   

                                                                                                                              \
                                                                                                                               
                                                                                                                               
                                                                                                               PostalAddress   
                                                                                                                    CityName   
                                                                                                                               
                                                                                                                               
                                                                                                                               
                                                                                                                               
                                                                                                                               
                          

The two columns below (built from a *path* by means of `dlsproc.xml.assemble_name`) are the most promising ones

In [ ]:
levels_entity_columns = [
    ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'],
    ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'Contact', 'Name']
]
# entity_columns = [dlsproc.xml.assemble_name(e) for e in levels_entity_columns]
# entity_columns

A `pd.DataFrame` with them is assembled

In [ ]:
names_df = insiders_df.loc[:, [dlsproc.hier.pad_col_levels(insiders_df, c) for c in levels_entity_columns]]
names_df

ContractFolderStatus  \
                                                                                                                                       LocatedContractingParty   
                                                                                                                                                         Party   
                                                                                                                                                     PartyName   
                                                                                                                                                          Name   
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
zip                                                file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom      0      Alcaldía del Ayuntamiento de Fresno de Cantespino   
                                                                                                      1      Consejo de Administración de ICEX España Expor...   
                                                                                                      2             Alcaldia del Ayuntamiento de Palma del Río   
                                                                                                      3      Junta de Gobierno del Ayuntamiento de Palma de...   
                                                                                                      4      Concejalía de Personal y Contratación del Ayun...   
..                                                                                                                                                         ...   
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3_202201... 492    Instituto Balear de Infraestructuras y Servici...   
                                                                                                      493        Servicio Murciano de Salud - Junta de Compras   
                                                                                                      494    Dirección Provincial del Servicio Público de E...   
                                                                                                      495    Junta de Gobierno Local del Ayuntamiento de Ca...   
                                                                                                      496    Alcaldía del Ayuntamiento de Oliva de la Frontera   

                                                                                                                                                                
                                                                                        

Are they identical?

In [ ]:
(names_df.iloc[:, 0] == names_df.iloc[:, 1]).all()

True

This column is used from now on

In [ ]:
entity_col = dlsproc.hier.pad_col_levels(insiders_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'])
entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '',
 '',
 '',
 '',
 '',
 '',
 '')

In [ ]:
insiders_df[entity_col]

zip                                                  file name                                                      entry
licitacionesPerfilesContratanteCompleto3_202203.zip  licitacionesPerfilesContratanteCompleto3.atom                  0        Alcaldía del Ayuntamiento de Fresno de Cantespino
                                                                                                                    1        Consejo de Administración de ICEX España Expor...
                                                                                                                    2               Alcaldia del Ayuntamiento de Palma del Río
                                                                                                                    3        Junta de Gobierno del Ayuntamiento de Palma de...
                                                                                                                    4        Concejalía de Personal y Contratación del Ayun...
   

### Amounts

Columns in the `pd.DataFrame` containing "Amount":

In [ ]:
insiders_df[dlsproc.hier.columns_containing(insiders_df, 'Amount')]

ContractFolderStatus  \
                                                                                                                        ProcurementProject   
                                                                                                                              BudgetAmount   
                                                                                                            EstimatedOverallContractAmount   
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
zip                                                file name                                          entry                                  
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3.atom      0                            6500.00   
                                                                                                      1                           50000.00   
                                                                                                      2                           51073.77   
                                                                                                      3                          171290.01   
                                                                                                      4                          148760.33   
..                                                                                                                                     ...   
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3_202201... 492                        317877.49   
                                                                                                      493                        708952.50   
                                                                                                      494                        140842.88   
                                                                                                      495                        296783.26   
                                                                                                      496                         51112.99   

                                                                                                                         \
                                                                                                                          
                                                                                                                          
                                                                                                            TotalAmount   
                                                                                                                          
                                                                                              

Let us focus on this one

In [ ]:
amount_col = dlsproc.hier.pad_col_levels(insiders_df, ['ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TotalAmount'])
amount_col

('ContractFolderStatus',
 'ProcurementProject',
 'BudgetAmount',
 'TotalAmount',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '')

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

The first few values in it

In [ ]:
insiders_df[amount_col].head()

zip                                                  file name                                      entry
licitacionesPerfilesContratanteCompleto3_202203.zip  licitacionesPerfilesContratanteCompleto3.atom  0          6500.00
                                                                                                    1         30250.00
                                                                                                    2         61799.26
                                                                                                    3        207260.91
                                                                                                    4         45000.00
Name: (ContractFolderStatus, ProcurementProject, BudgetAmount, TotalAmount, , , , , , , , ), dtype: float64

A *grouped* `pd.DataFrame` showing the number of contracts and overall amount for each entity

In [ ]:
number_amount_df = insiders_df[[entity_col, amount_col]].groupby(entity_col).agg(amount=(amount_col, 'sum'), number=(amount_col, 'count'))
number_amount_df

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , , , , , , )",,
Junta de Gobierno de Val de San Vicente,4.366600e+04,2
13ª Zona de la Guardia Civil - Cantabria,1.584000e+04,2
5ª Zona de la Guardia Civil (Murcia),2.706165e+04,1
8ª Zona de la Guardia Civil Comunidad Autónoma...,0.000000e+00,4
ADIF - Consejo de Administración,4.556022e+09,147
...,...,...
"Área de Gobierno de Urbanismo, Edificación y S...",6.086841e+06,15
Área de Gobierno de Vicealcaldía,8.683304e+07,21
Área de Innovación y Digitalización Urbana del...,2.121195e+05,3


Strangely enough, there are entities with contracts amounting overall to $0$

In [ ]:
number_amount_df.iloc[3]

amount    0.0
number    4.0
Name: 8ª Zona de la Guardia Civil Comunidad Autónoma de Aragón, dtype: float64

In [ ]:
insiders_df.loc[insiders_df[entity_col] == '8ª Zona de la Guardia Civil Comunidad Autónoma de Aragón', :]

id  \
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
zip                                                file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3_202203... 476    https://contrataciondelestado.es/sindicacion/l...   
                                                   licitacionesPerfilesContratanteCompleto3_202203... 130    https://contrataciondelestado.es/sindicacion/l...   
                                                   licitacionesPerfilesContratanteCompleto3_202203... 62     https://contrataciondelestado.es/sindicacion/l...   
                                                   licitacionesPerfilesContratanteCompleto3_202203... 21     https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                       summary  \
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                         

Entities with the biggest amounts

In [ ]:
number_amount_df.sort_values('amount', ascending=False)

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , , , , , , )",,
Dirección General de la Mutualidad de Funciona...,1.066429e+10,5
ADIF Alta Velocidad - Consejo de Administración,5.273203e+09,67
Dirección del Instituto Nacional de Gestión Sa...,4.660165e+09,65
ADIF - Consejo de Administración,4.556022e+09,147
Aena. Consejo de Administración,4.359473e+09,112
...,...,...
Alcaldía del Ayuntamiento de Villaornate y Castro,0.000000e+00,1
Delegación Provincial de la Consejería de Desa...,0.000000e+00,2
Alcaldía del Ayuntamiento de Bugedo,0.000000e+00,2


### Documents

Columns containing *DocumentReference* in the name, allegedly URLs to documents

In [ ]:
documents_df = insiders_df.loc[:, dlsproc.hier.columns_containing(insiders_df, 'DocumentReference')].droplevel(['zip', 'file name'])
documents_df.head()

ContractFolderStatus  \
                               TenderingTerms   
      ProcurementLegislationDocumentReference   
                                           ID   
                                                
                                                
                                                
                                                
                                                
                                                
                                                
                                                
entry                                           
0                                         N/A   
1                                  2014/24/EU   
2                                  2014/24/EU   
3                                  2014/24/EU   
4                                  2014/24/EU   

                                                          \
                                  LegalDocumentReference   
                                                      ID   
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
entry                                                      
0                                               PCAP.pdf   
1                                                   <NA>   
2      Pliego Clausulas Administrativas Particulares.pdf   
3                   Pliego Clausulas Administrativas.pdf   
4                                     7 PCAP_SM22_21.pdf   

                                                          \
                                                           
                                              Attachment   
                                       ExternalReference   
                                                     URI   
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
entry                                                      
0      https://contrataciondelestado.es/wps/wcm/conne...   
1                                                   <NA>   
2      https://contrataciondelestado.es/wps/wcm/conne...   
3      https://contrataciondelestado.es/wps/wcm/conne...   
4      https://contrataciondelestado.es/wps/wcm/conne...   

                                                                         \
                                             TechnicalDocumentReference   
                                                                     ID   
                                                                          
                       DocumentHash                                       
                                                                          
                                                                          
                                                                          
                                                                          
                                                                          
                                                                          
                                                                          
entry                                                                     
0    

Columns that have been captured

In [ ]:
documents_df.columns.values

array([('ContractFolderStatus', 'TenderingTerms', 'ProcurementLegislationDocumentReference', 'ID', '', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LegalDocumentReference', 'ID', '', '', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LegalDocumentReference', 'Attachment', 'ExternalReference', 'URI', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LegalDocumentReference', 'Attachment', 'ExternalReference', 'DocumentHash', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'TechnicalDocumentReference', 'ID', '', '', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'TechnicalDocumentReference', 'Attachment', 'ExternalReference', 'URI', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'TechnicalDocumentReference', 'Attachment', 'ExternalReference', 'DocumentHash', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'AdditionalDocumentReference', 'ID', '', '', '', '', '', '', '', '', ''),
       ('Contr

Let us check one of those ending in a *leaf* named *URI*

In [ ]:
documents_df[
    dlsproc.hier.pad_col_levels(documents_df, ('ContractFolderStatus', 'GeneralDocument', 'GeneralDocumentDocumentReference', 'Attachment', 'ExternalReference', 'URI'))
].iloc[7]

'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=d704d108-6a36-4c60-ad5b-123954b6d913'

## Outsiders

> Licitaciones publicadas en la Plataforma mediante mecanismos de agregación, excluyendo los contratos menores

A file to *cache* the read data

In [ ]:
outsiders_output_file = dlsproc.io.File('outsiders')

The directory for *insiders* data

In [ ]:
outsiders_directory = data_directory / 'agregados'
assert insiders_directory.exists()
print(insiders_directory)

/home/manu/dlsproc/data/perfiles_plataforma


The data is read from a file if present, or from the given directory if not

In [ ]:
if outsiders_output_file.exists():
    outsiders_df = outsiders_output_file.read()
else:
    # data is read from the given directory, and concatenated into a single `pd.DataFrame`...
    outsiders_df = dlsproc.bundle.read_zips_directory(outsiders_directory, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    outsiders_df = dlsproc.hier.flat_df_to_multiindexed_df(outsiders_df)
    
    # ...and saved
    outsiders_output_file.write(outsiders_df)

How many entries and how many fields?

In [ ]:
outsiders_df.shape

(49422, 40)

Examples of columns

In [ ]:
outsiders_df.columns.values[10:20]

array([('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'EstimatedOverallContractAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RequiredCommodityClassification', 'ItemClassificationCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RealizedLocation', 'CountrySubentityCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'PlannedPeriod', 'DurationMeasure', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'ProcedureCode', '', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndDate', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndTime', '', '', ''),
       ('ContractFolderStatus', 'ValidNoticeInfo', 'NoticeTypeCode', '', '', '', ''),
       ('ContractFolderStatus', 'ValidNotice

The first few rows

In [ ]:
outsiders_df.head()

id  \
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
zip                                       file name                           entry                                                      
PlataformasAgregadasSinMenores_202201.zip PlataformasAgregadasSinMenores.atom 0      https://contrataciondelestado.es/sindicacion/P...   
                                                                              1      https://contrataciondelestado.es/sindicacion/P...   
                                                                              2      https://contrataciondelestado.es/sindicacion/P...   
                                                                              3      https://contrataciondelestado.es/sindicacion/P...   
                                                                              4      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                               summary  \
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
zip                                       file name                           entry                                                      
PlataformasAgregadasSinMenores_202201.zip PlataformasAgregadasSinMenores.atom 0      Id licitación: CONTR 2021 0000416750; Órgano d...   
                                                                              1      Id licitación: 8023_1/2022; Órgano de Contrata...   
                                                                              2      Id licitación: 1009_00001119-2021-0031; Órgano...   
                                                                              3      Id licitación: 3064_23/2021; Órgano de Contrat...   
                                                                              4      Id licitación: 451/2021; Órgano de Contratació...   

                                                                                                                                 title  \
                                                                                                                                         
                                                                                                                                         
                                                                                                                                

Some types

In [ ]:
outsiders_df.dtypes[:20]

id                                                                                                                                        string
summary                                                                                                                                   string
title                                                                                                                                     string
updated                                                                                                                      datetime64[ns, UTC]
ContractFolderStatus  ContractFolderID                                                                                                    string
                      ContractFolderStatusCode                                                                                            string
                      LocatedContractingParty   Party                            PartyName                       Name             

### Entity

Columns in the `pd.DataFrame` containing "Name" in their names are displayed

In [ ]:
outsiders_df[dlsproc.hier.columns_containing(outsiders_df, 'Name')].columns.values

array([('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'ParentLocatedParty', 'PartyName', 'Name', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'Name', '', '', '', ''),
       ('ContractFolderStatus', 'ValidNoticeInfo', 'AdditionalPublicationStatus', 'PublicationMediaName', '', '', ''),
       ('ContractFolderStatus', 'TenderResult', 'WinningParty', 'PartyName', 'Name', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'ParentLocatedParty', 'ParentLocatedParty', 'PartyName', 'Name', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'ParentLocatedParty', 'ParentLocatedParty', 'ParentLocatedParty', 'PartyName', 'Name')],
      dtype=object)

In [ ]:
entity_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'])
entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '',
 '')

In [ ]:
outsiders_df[entity_col]

zip                                        file name                                            entry
PlataformasAgregadasSinMenores_202201.zip  PlataformasAgregadasSinMenores.atom                  0        Dirección Provincial del Servicio Andaluz de E...
                                                                                                1                                        Ayuntamiento Anue
                                                                                                2                      Dirección General de Medio Ambiente
                                                                                                3                                  Junta de Gobierno Local
                                                                                                4                                   Ajuntament de Sabadell
                                                                                                                           

### Amounts

Columns in the `pd.DataFrame` containing "Amount":

In [ ]:
outsiders_df[dlsproc.hier.columns_containing(outsiders_df, 'Amount')].columns.values

array([('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'EstimatedOverallContractAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount', '', '', ''),
       ('ContractFolderStatus', 'TenderResult', 'AwardedTenderedProject', 'LegalMonetaryTotal', 'TaxExclusiveAmount', '', '')],
      dtype=object)

In [ ]:
amount_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount'])
amount_col

('ContractFolderStatus',
 'ProcurementProject',
 'BudgetAmount',
 'TaxExclusiveAmount',
 '',
 '',
 '')

In [ ]:
number_amount_df = outsiders_df[[entity_col, amount_col]].groupby(entity_col).agg(amount=(amount_col, 'sum'), number=(amount_col, 'count'))
number_amount_df

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
AGENCIA NAVARRA DE AUTONOMÍA Y DESARROLLO DE L...,2.456951e+06,2
AJUNTAMENT DE FALSET,1.646735e+05,5
AJUNTAMENT DE SANT JAUME DELS DOMENYS,1.044863e+07,8
AJUNTAMENT DE SANTA FE DEL PENEDÈS,3.315576e+05,3
AJUNTAMENT DE VILA-RODONA,6.023343e+05,4
...,...,...
Área de Gobierno de Seguridad y Emergencias,3.944037e+06,6
Área de Gobierno de Urbanismo y Vivienda,1.073582e+09,253
"Área de Gobierno de las Artes, Deportes y Turismo",4.975287e+05,15


Entities with the biggest amounts

In [ ]:
number_amount_df.sort_values('amount', ascending=False)

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
Consejería de Transportes e Infraestructuras,6.154296e+09,129
Àrea Metropolitana de Barcelona,4.974587e+09,370
"Canal de Isabel II, S.A.",3.813725e+09,662
Servicio Andaluz de Salud,3.798085e+09,760
"Consejería de Familia, Juventud y Política Social",2.670492e+09,216
...,...,...
Eurorregión Nueva-Aquitania Euskadi Navarra,0.000000e+00,0
Jefa de la Sección de Contratación y Seguros.,0.000000e+00,0
Asamblea General de la Mancomunidad de la Coma...,0.000000e+00,0


### Documents

Columns containing *DocumentReference* in the name, allegedly URLs to documents

In [ ]:
outsiders_df.loc[:, dlsproc.hier.columns_containing(outsiders_df, 'DocumentReference')].droplevel(['zip', 'file name'])

ContractFolderStatus  \
                             ValidNoticeInfo   
                 AdditionalPublicationStatus   
      AdditionalPublicationDocumentReference   
                                   IssueDate   
                                               
                                               
entry                                          
0                                 2022-01-11   
1                                 2022-02-01   
2                                 2022-01-31   
3                                 2022-01-31   
4                                 2022-01-10   
..                                       ...   
387                               2022-02-28   
388                               2022-02-28   
389                               2022-01-20   
390                               2022-02-04   
391                               2022-02-28   

                                              \
                      LegalDocumentReference   
                                          ID   
                                               
                                               
                                               
                                               
entry                                          
0                                       <NA>   
1                                       <NA>   
2                                       <NA>   
3                                       <NA>   
4      Plec de clausules administratives.pdf   
..                                       ...   
387                        1354909934663.pdf   
388                        1354876656292.pdf   
389                        1354909879660.pdf   
390                        1354913707024.pdf   
391                                     <NA>   

                                                          \
                                                           
                                              Attachment   
                                       ExternalReference   
                                                     URI   
                                                           
                                                           
entry                                                      
0                                                   <NA>   
1                                                   <NA>   
2                                                   <NA>   
3                                                   <NA>   
4      https://contractaciopublica.gencat.cat/ecofin_...   
..                                                   ...   
387    http://www.madrid.org/contratos-publicos/13549...   
388    http://www.madrid.org/contratos-publicos/13548...   
389    http://www.madrid.org/contratos-publicos/13549...   
390    http://www.madrid.org/contratos-publicos/13549...   
391                                                 <NA>   

                                  \
      TechnicalDocumentReference   
                              ID   
                                   
                                   
                                   
                                   
entry                              
0                           <NA>   
1                           <NA>   
2                           <NA>   
3                           <NA>   
4                   Projecte.pdf   
..                           ...   
387            1354909933889.pdf   
388            1354876187519.pdf   
389            1354908647502.pdf   
390            1354913707812.pdf   
391                         <NA>   

                                                          
                                                          
                                              Attachment  
                                       ExternalReference  
                                                     URI  
                                                          
                  

In [ ]:
# outsiders_df.droplevel(['zip', 'file name']).iloc[50].index
# outsiders_df.iloc[50].index

### Questions

A particular entry

In [ ]:
# outsiders_df.droplevel(['zip', 'file name']).iloc[25000]
outsiders_df.iloc[25000]

id                                                                                                                                                                    https://contrataciondelestado.es/sindicacion/P...
summary                                                                                                                                                               Id licitación: 1340; Órgano de Contratación: ...
title                                                                                                                                                                 Asistencia técnica para formación y tutorías d...
updated                                                                                                                                                                                2022-03-16 15:55:32.142000+00:00
ContractFolderStatus  ContractFolderID                                                                                                  

How many non-null rows provide a value for the entity URL?

In [ ]:
outsiders_df[('ContractFolderStatus', 'LocatedContractingParty', 'BuyerProfileURIID', '', '', '', '')].notna().sum()

31201

Grouping exploting the entity URL

In [ ]:
outsiders_df.groupby(('ContractFolderStatus', 'LocatedContractingParty', 'BuyerProfileURIID', '', '', '', '')).size().sort_values()

(ContractFolderStatus, LocatedContractingParty, BuyerProfileURIID, , , , )
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
                                                     ... 
https://contractaciopublica.gencat.cat/ecofin_...     551
https://contractaciopublica.gencat.cat/ecofin_...     569
http://www.madrid.org/cs/Satellite?cid=1205761...     618
https://contractaciopublica.gencat.cat/ecofin_...     784
http://www.madrid.org/cs/Satellite?cid=1204201...    5863
Length: 1255, dtype: int64